In [1]:
from src.loader import InstanceLoader
obj_folder = "/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/segmentation"

In [2]:
t = InstanceLoader(obj_folder=obj_folder, skeleton_toml_path="skeleton.toml", pose_csv="./tracking/28/240905-1616_session28_track99_points3d.csv", obj_ref_frame=2599, obj_transform_toml=f"{obj_folder}/240905-1616_seed_session28_first.toml")

In [3]:
from src.animal import AnimalList
track_folder = "/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/tracking"
# A = AnimalList("skeleton.toml", track_folder, session_number=28)
t = InstanceLoader(obj_folder=obj_folder,
                        skeleton_toml_path="skeleton.toml",
                        track_folder=track_folder,
                        session_number=28,
                        obj_ref_frame=2599,
                        obj_transform_toml=f"{obj_folder}/240905-1616_seed_session28_first.toml")

PosixPath('/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/tracking/240905-1616_session28_track13_points3d.csv') generated an exception: Pose csv does not contain any valid frames
PosixPath('/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/tracking/240905-1616_session28_track12_points3d.csv') generated an exception: Pose csv does not contain any valid frames
PosixPath('/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/tracking/240905-1616_session28_track11_points3d.csv') generated an exception: Pose csv does not contain any valid frames
PosixPath('/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/tracking/240905-1616_session28_track10_points3d.csv') generated an exception: Pose csv does not contain any valid frames
PosixPath('/Users/persie/PhD_Code/3d_ant_data_rle/240905-1616/outputs/tracking/240905-1616_session28_track0_points3d.csv') generated an exception: Pose csv does not contain any valid frames
PosixPath('/Users/persie/PhD_Code/3d_ant_data_

In [3]:
from src.collision import CollisionDetector
c = CollisionDetector(t)

In [6]:
c._collision_df.reset_index(inplace=True)
c._collision_df.set_index(["Frame", "Track"], inplace=True)

In [4]:
from src.visual_widget import MultiViewer
V = MultiViewer(t.animal_list, t.obj_list, 0)

2025-04-16 17:09:52.081 python[46912:16400625] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-16 17:09:52.115 python[46912:16400625] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [ ]:
from scripts.tools.x_y_z_plot import KPPosition, KPVelocityXYZ, KPAccXYZ
from scripts.tools.magnitude_plot import KPVelocityMag

KP = KPAccXYZ(t.animal, node=['a_L2', 'neck'], window_size=1)